In [ ]:
!pip install rectools[all]

In [ ]:
from rectools.dataset import Dataset
from rectools import Columns
from rectools.metrics import MAP
import matplotlib.pyplot as plt
import numpy as np

# **Data preprocessing**

In [1]:
import pandas as pd
import numpy as np
df = pd.read_table('/kaggle/input/ok-dataset/train_df.tsv')

In [2]:
def pad_to_length(array, length=5):
    if len(array) < length:
        padding = np.full(length - len(array), np.nan)
        return np.concatenate((array, padding))
    return array

def status_to_weight(staus):
    if staus == 'A': # полноценный участник
        weight = 1
    if staus == 'P': # ушел из группы, но может вернуться 
        weight = 0.25
    if staus == 'B': # был заблокирован 
        weight = -1
    if staus == 'M': # модератор 
        weight = 1.5
    if staus == '!': # администратор
        weight = 2
    if staus == 'Y': # возможный участник мероприятия (не понял, что они конкретно иммели ввиду), 
        weight = 1.25 #но как я понял это, что он участник, который записался на что-то дополнительное   
    if staus == 'I': # Человека пригласили в группу
        weight = 0.5 
    if staus == 'R': # Пользователь отправил запрос на вступление
        weight = 1
    if staus == 'F': # Следит за группой, но не учавствует
        weight = 0.9 
        

def status_to_weight(staus):
    if staus == 'A': # полноценный участник
        weight = 1.
    if staus == 'P': # ушел из группы, но может вернуться 
        weight = -1.
    if staus == 'B': # был заблокирован 
        weight = -1.
    if staus == 'M': # модератор 
        weight = 1.
    if staus == '!': # администратор
        weight = 1.
    if staus == 'Y': # возможный участник мероприятия (не понял, что они конкретно иммели ввиду), 
        weight = 1. #но как я понял это, что он участник, который записался на что-то дополнительное   
    if staus == 'I': # Человека пригласили в группу
        weight = 1.
    if staus == 'R': # Пользователь отправил запрос на вступление
        weight = 1
    if staus == 'F': # Следит за группой, но не учавствует
        weight = 1.

        
def cut_last(feature): # решение проблем моей криворускости)))
    return '_'.join(feature.split('_')[:-1])

In [ ]:
df.isnull().sum()

In [ ]:
mean_customer_in_group_per_user = df.groupby('customer_id')['customers_count'].mean().items()
mean_customer_in_group_per_user = {index: value for index, value in mean_customer_in_group_per_user}

mean_messages_in_group_per_user = df.groupby('customer_id')['messages_count'].mean().items()
mean_messages_in_group_per_user = {index: value for index, value in mean_messages_in_group_per_user}

customer_most_poppular_type = df.groupby('customer_id')['type'].agg(lambda type: type.mode()[0])
customer_most_poppular_type = {index: value for index, value in customer_most_poppular_type.items()}

customer_most_popular_region = df.groupby('customer_id')['region_id'].agg(lambda type: type.mode()[0] if not type.mode().empty else np.nan)
customer_most_popular_region = {index: value for index, value in customer_most_popular_region.items()}

n = 5
customer_most_popular_themeid = df.groupby('customer_id')['themeid'].agg(lambda type: pad_to_length(type.value_counts().index[:n].tolist()) if not type.mode().empty else [np.nan for _ in range(n)])
customer_most_popular_themeid = {index: value for index, value in customer_most_popular_themeid.items()}

n = 3
customer_most_popular_business_category = df.groupby('customer_id')['business_category'].agg(lambda type: pad_to_length(type.value_counts().index[:n].tolist()) if not type.mode().empty else [np.nan for _ in range(n)])
customer_most_popular_business_category = {index: value for index, value in customer_most_popular_business_category.items()}

n = 3
customer_most_popular_business_parent = df.groupby('customer_id')['business_parent'].agg(lambda type: pad_to_length(type.value_counts().index[:n].tolist()) if not type.mode().empty else [np.nan for _ in range(n)])
customer_most_popular_business_parent = {index: value for index, value in customer_most_popular_business_parent.items()}


users_features = pd.DataFrame({'customer_id': df.customer_id.drop_duplicates().values})


users_features['mean_customers'] = users_features.customer_id.apply(lambda customer_id: mean_customer_in_group_per_user[customer_id])
users_features['mean_messages'] = users_features.customer_id.apply(lambda customer_id: mean_messages_in_group_per_user[customer_id])
users_features['most_popular_type'] = users_features.customer_id.apply(lambda customer_id: customer_most_poppular_type[customer_id])
users_features['most_popular_region'] = users_features.customer_id.apply(lambda customer_id: customer_most_popular_region[customer_id])

n = 5
for counter in range(n):
    users_features[('most_popular_theme_' + str(counter))] = users_features.customer_id.apply(lambda customer_id: customer_most_popular_themeid[customer_id][counter])
n = 3
for counter in range(n):
    users_features[('most_popular_business_category_' + str(counter))] = users_features.customer_id.apply(lambda customer_id: customer_most_popular_business_category[customer_id][counter])
n = 3
for counter in range(n):
    users_features[('most_popular_business_parent_' + str(counter))] = users_features.customer_id.apply(lambda customer_id: customer_most_popular_business_parent[customer_id][counter])

users_features = pd.melt(users_features, id_vars = 'customer_id')   
users_features = users_features.sort_values(by=['customer_id', 'variable'])
users_features = users_features.reset_index(drop=True)
users_features = users_features.rename(columns={"customer_id": Columns.User, "variable": "feature"})


items_features = df.copy()
items_features = items_features.drop_duplicates(subset=["community_id"])
items_features = items_features.drop(columns=['customer_id', 'join_request_date', 'status', 'Unnamed: 0'])
items_features = pd.melt(items_features, id_vars = 'community_id')
items_features = items_features.sort_values(by=['community_id'])
items_features = items_features.reset_index(drop=True)
items_features = items_features.rename(columns={"community_id": Columns.Item, "variable": "feature"})

In [ ]:
train_dataset = df.copy()
train_dataset = train_dataset[['community_id', 'customer_id', 'status', 'join_request_date']]

In [ ]:
train_dataset

In [ ]:
#weights = train_dataset.status.apply(lambda status: status_to_weight(status))
weights = train_dataset.status.apply(lambda status: status_to_weight(str(status)))

In [ ]:
weights